RF1 — Alta y gestión de libros 

Registrar libros con atributos mínimos: isbn, titulo, autor, ejemplares_totales. 
Consultar disponibilidad (ejemplares_disponibles). 
Buscar por título o autor. 

## Alumno : Najera Beltran Bryan Alexis
## Horario : 12-13

In [10]:
# Defino una clase llamada libro
class libro():
    
    # Este es el constructor, se ejecuta cuando creo un objeto libro
    def __init__(self, isbn, titulo, autor, ejemplares):
        self.isbn = isbn              # Guardamos el ISBN del libro
        self.titulo = titulo          # Guardamos el título del libro
        self.autor = autor            # Guardamos el autor del libro
        self.ejemplares = ejemplares  # Cantidad de libros disponibles
        self.librosPrestado = 0       # Al inicio no hay libros prestados

    # Método para ver cuántos ejemplares hay disponibles
    def ejemplares_disponibles(self):
        return self.ejemplares

    # Método que revisa si hay al menos un libro disponible
    def disponible(self):
        return self.ejemplares > 0   # Regresa True o False

    # Método para prestar un libro 
    def reducir_ejemplares(self):
        if self.ejemplares > 0:          # Verificamos que haya libros
            self.ejemplares -= 1         # Restamos un ejemplar
            self.librosPrestado += 1     # Sumamos uno a los prestados
            return True                 # Se pudo prestar
        return False                    # No había libros disponibles

    # Método para devolver un libro
    def devolver_libro(self):
        if self.librosPrestado > 0:      # Verificamos que haya libros prestados
            self.ejemplares += 1         # Regresamos un ejemplar
            self.librosPrestado -= 1     # Restamos uno de prestados
            return True                 # Se devolvió correctamente
        return False                    # No había libros para devolver

In [11]:
libro1=libro(12932,'chuyito','javier',2)

In [12]:
libro1.reducir_ejemplares()


True

In [13]:
libro1.ejemplares_disponibles()

1

In [14]:
libro1.devolver_libro()

True

RF2 — Registro de usuarios 

Alta de usuario con id_usuario (único) y nombre. 
Consultar usuarios existentes. 

In [15]:
# Defino una clase llamada usuario
class usuario():
    
    # Constructor: se ejecuta cuando creo un nuevo usuario
    def __init__(self, id_usuario, nombre):
        self.id_usuario = id_usuario          # ID único del usuario
        self.nombre = nombre                  # Nombre del usuario
        self.prestamos_activos = []           # Lista de libros prestados (ISBN)

    # Método para verificar si puede pedir más libros
    def puede_pedir_prestado(self):
        # Solo puede tener máximo 3 libros prestados
        return len(self.prestamos_activos) < 3

    # Método para registrar un préstamo
    def registrar_prestamo(self, isbn):
        # Primero verifico si puede pedir prestado
        if self.puede_pedir_prestado():
            self.prestamos_activos.append(isbn)  # Agrego el ISBN a la lista
            return True                          # Préstamo exitoso
        return False                             # No puede pedir más libros

    # Método para registrar la devolución de un libro
    def registrar_devolucion(self, isbn):
        # Verifico si ese libro está en la lista de préstamos
        if isbn in self.prestamos_activos:
            self.prestamos_activos.remove(isbn)  # Elimino el ISBN de la lista
            return True                          # Devolución exitosa
        return False                             # Ese libro no estaba prestado

In [16]:
# Clase para manejar la biblioteca
class GestionBiblioteca:

    # Constructor: se ejecuta cuando se crea el objeto
    def __init__(self, libros, usuarios):
        self.libros = libros            # guardo la lista de libros
        self.usuarios = usuarios        # guardo la lista de usuarios
        self.prestamos_registro = []    # lista para guardar préstamos realizados
        self.sig_id = len(usuarios) + 1 # siguiente ID disponible para usuarios

    # Método para buscar un libro por su ISBN
    def buscar_libro(self, isbn):
        for l in self.libros:           # recorro todos los libros
            if l.isbn == isbn:          # si el ISBN coincide
                return l                # regreso el libro encontrado
        return None                     # si no lo encuentra regreso None

    # Método para buscar un usuario por su ID
    def buscar_usuario(self, id_u):
        for u in self.usuarios:         # recorro todos los usuarios
            if u.id_usuario == id_u:    # si el ID coincide
                return u                # regreso el usuario
        return None                     # si no existe regreso None

    # Método para prestar un libro
    def realizar_prestamo(self, isbn, id_u, fecha):
        libro = self.buscar_libro(isbn)     # busco el libro
        usuario = self.buscar_usuario(id_u) # busco el usuario

        # verifico si existen
        if libro == None:
            return "No existe el libro"
        if usuario == None:
            return "No existe el usuario"

        # verifico si el usuario ya tiene ese libro
        if isbn in usuario.prestamos_activos:
            return "Ya tiene ese libro"

        # verifico si puede pedir más libros
        if not usuario.puede_pedir_prestado():
            return "Ya tiene muchos libros"

        # intento prestar el libro (reducir ejemplares)
        if libro.reducir_ejemplares():
            usuario.registrar_prestamo(isbn)              # guardo el préstamo en el usuario
            self.prestamos_registro.append((isbn, id_u, fecha)) # guardo en historial
            return "Prestado"
        else:
            return "No hay libros"                        # no había disponibles

    # Método para devolver un libro
    def realizar_devolucion(self, isbn, id_u):
        libro = self.buscar_libro(isbn)     # busco el libro
        usuario = self.buscar_usuario(id_u) # busco el usuario

        # si alguno no existe
        if libro == None or usuario == None:
            return "Error"

        # intento devolver el libro
        if usuario.registrar_devolucion(isbn):
            libro.devolver_libro()          # aumento ejemplares
            return "Devuelto"
        else:
            return "No lo tenia"            # el usuario no tenía ese libro

    # Método para obtener los 3 libros más prestados
    def top_3_prestados(self):
        lista = self.libros.copy()          # hago una copia de la lista

        # ordeno la lista (de mayor a menor)
        for i in range(len(lista)):
            for j in range(len(lista)):
                if lista[i].librosPrestado > lista[j].librosPrestado:
                    aux = lista[i]          # intercambio posiciones
                    lista[i] = lista[j]
                    lista[j] = aux

        return lista[0:3]                   # regreso los primeros 3

In [17]:
biblio = [
    libro(12932, 'Cien Años de Soledad', 'García Márquez', 2),
    libro(99541, 'El Quijote', 'Cervantes', 1),
    libro(77777, 'Rayuela', 'Cortázar', 3)
]

usuarios_init = [
    usuario(1, 'Ana'),
    usuario(2, 'Luis')
]

gestionador = GestionBiblioteca(biblio, usuarios_init)



In [18]:
# Ciclo infinito para que el menú se repita siempre
while True:
    
    # Mostrar el menú de opciones
    print("\n1 Alta usuario")  # Opción para agregar usuario
    print("2 Listar usuarios")  # Ver usuarios registrados
    print("3 Prestar libro")  # Prestar un libro
    print("4 Devolver libro")  # Devolver un libro
    print("5 Top 3 libros más prestados")  # Ver los más prestados
    print("6 Agregar libro")  # Agregar un libro nuevo
    print("7 listar libros")  # Mostrar libros
    print("8 Salir")  # Salir del programa

    # Pedir al usuario que elija una opción
    op = input("Opcion: ")

    # Si elige la opción 1
    if op == "1":
        nom = input("Nombre: ")  # Pedir nombre
        nuevo = usuario(gestionador.sig_id, nom)  # Crear usuario
        gestionador.usuarios.append(nuevo)  # Guardarlo en la lista
        print("ID asignado:", gestionador.sig_id)  # Mostrar ID
        gestionador.sig_id += 1  # Aumentar el ID para el siguiente usuario

    # Opción 2: mostrar usuarios
    elif op == "2":
        for u in gestionador.usuarios:  # Recorrer lista de usuarios
            print(u.id_usuario, u.nombre, "Prestamos:", len(u.prestamos_activos))

    # Opción 3: prestar libro
    elif op == "3":
        isbn = int(input("ISBN: "))  # Pedir ISBN
        uid = int(input("ID usuario: "))  # Pedir ID usuario
        fecha = input("Fecha: ")  # Pedir fecha
        print(gestionador.realizar_prestamo(isbn, uid, fecha))  # Hacer préstamo

    # Opción 4: devolver libro
    elif op == "4":
        isbn = int(input("ISBN: "))
        uid = int(input("ID usuario: "))
        print(gestionador.realizar_devolucion(isbn, uid))  # Hacer devolución

    # Opción 5: mostrar top 3 libros más prestados
    elif op == "5":
        top = gestionador.top_3_prestados()  # Obtener lista
        i = 1  # Contador
        for l in top:
            print(i, l.titulo, "-", l.librosPrestado, "prestamos")
            i += 1  # Aumentar contador

    # Opción 6: agregar libro
    elif op == "6":
        isbn = int(input("ISBN: "))
        titulo = input("Titulo: ")
        autor = input("Autor: ")
        ejemplares = int(input("Cantidad de ejemplares: "))

        nuevo_libro = libro(isbn, titulo, autor, ejemplares)  # Crear libro
        gestionador.libros.append(nuevo_libro)  # Guardar libro

        print("Libro agregado correctamente")

    # Opción 7: listar libros
    elif op == "7":
        for l in gestionador.libros:  # Recorrer libros
            print(l.isbn, l.titulo, l.autor, "Disponibles:", l.ejemplares)

    # Opción 8: salir del programa
    elif op == "8":
        break  # Termina el ciclo

    # Si elige una opción incorrecta
    else:
        print("No valido")


1 Alta usuario
2 Listar usuarios
3 Prestar libro
4 Devolver libro
5 Top 3 libros más prestados
6 Agregar libro
7 listar libros
8 Salir
12932 Cien Años de Soledad García Márquez Disponibles: 2
99541 El Quijote Cervantes Disponibles: 1
77777 Rayuela Cortázar Disponibles: 3

1 Alta usuario
2 Listar usuarios
3 Prestar libro
4 Devolver libro
5 Top 3 libros más prestados
6 Agregar libro
7 listar libros
8 Salir
